In [7]:
import csv
import os
import requests
import json
import time
import io
import datetime; 
import pandas as pd
from tqdm import tqdm
from threading import Thread, Event
import time
from pandas import ExcelWriter

stop_it = Event()

os.chdir(r"D:\Users\snangare\Sandesh\API")
error_log_list = []
success_log_list = []

def authenticate():
    
    endpoint = 'https://amgensbx-etmf.veevavault.com/api/v20.3/auth' 
    parameters = {
        'username':'snangare@amgensbx.com',
        'password':'A1ph@12345'
    }
    
    try: 
        response = requests.post(endpoint, data=parameters).json()
        if response['responseStatus'] != 'SUCCESS':
            raise Exception()
        else:
            return response['sessionId']
    except:
        error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
        print (error_message)

        
def fetch_documents(session_id,endpoint,doc_name,doc_id):
    
    headers = {
        'Authorization': session_id
    }

    try: 
        response = requests.get(endpoint, headers=headers).json()
        if response['responseStatus'] != 'SUCCESS':
            raise Exception()
        elif len(response['data'] ) == 0:
            error_log_list.append("Document with id "+doc_id+" had no attachment at time "+str(datetime.datetime.now())+" IST")

        else:
            
            count = 1
            
            for attachment in response['data']:
                
                url = attachment['versions'][0]['url'][:attachment['versions'][0]['url'].index('versions')]+"file"
                
                res = requests.get(url,headers=headers)
                
                with open(doc_name+'_attachment_'+str(count), 'wb') as f:
                    f.write(res.content)  
                
                extension = attachment['filename__v'][attachment['filename__v'].rindex('.'):]
                filename = attachment['filename__v'][:attachment['filename__v'].rindex('.')]
                

                os.rename(doc_name+'_attachment_'+str(count), doc_id+"_"+filename+"_"+str(attachment['id'])+'_Attachment_'+str(count)+extension)
                          
                success_log_list.append(doc_id+"_"+filename+"_"+str(attachment['id'])+'_Attachment_'+str(count)+extension+" exported successfully at "+str(datetime.datetime.now())+" IST")
            
                count = count + 1
    except:
        error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
        print (error_message)
        
def execute():
    session_id = authenticate()
    
    df = pd.read_csv('document_ids-attachment.csv')
    os.chdir("exports/attachments")
    
    for ind in tqdm(df.index):

        doc_id = df['Document ID'][ind]
        doc_name = str(df['Document Number'][ind])
        
        URI = 'https://amgensbx-etmf.veevavault.com/api/v20.3/'
        action = 'objects/documents/'+str(doc_id)+"/attachments"
        endpoint = URI + action
        
        stuff_doing_thread = Thread(target=fetch_documents,args=(session_id,endpoint,doc_name,str(doc_id)))
        stuff_doing_thread.start()
        stuff_doing_thread.join(timeout=20)
        stop_it.set()

        
    success_log_df = pd.DataFrame(success_log_list, columns =['Success Log'])    
    writer = ExcelWriter(r'D:\Users\snangare\Sandesh\API\logs\attachment\success_log.xlsx')
    success_log_df.to_excel(writer,'success_log',index=False)
    writer.save()
    
    error_log_df = pd.DataFrame(error_log_list, columns =['Error Log'])    
    writer = ExcelWriter(r'D:\Users\snangare\Sandesh\API\logs\attachment\error_log.xlsx')
    error_log_df.to_excel(writer,'error_log',index=False)
    writer.save()

execute()  

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:06<00:00,  1.75it/s]
